In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import ipywidgets
import bqplot

# Data Prep

In [2]:
buildings = pd.read_csv('building_inventory.csv')

In [3]:
buildings

,Agency Name,Location Name,Address,City,Zip code,County,Congress Dist,Congressional Full Name,Rep Dist,Rep Full Name,...,Bldg Status,Year Acquired,Year Constructed,Square Footage,Total Floors,Floors Above Grade,Floors Below Grade,Usage Description,Usage Description 2,Usage Description 3
0,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,1975,1975,144,1,1,0,Unusual,Unusual,Not provided
1,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
2,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
3,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
4,Department of Natural Resources,Anderson Lake Conservation Area - Fulton County,Anderson Lake C.a.,Astoria,61501,Fulton,17,Cheri Bustos,93,Hammond Norine K.,...,In Use,2004,2004,144,1,1,0,Unusual,Unusual,Not provided
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8857,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte. 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,0,0,432,1,0,0,Storage,NaN,NaN
8858,Department of Transportation,Belvidere Maintenance Storage Facility - Boone...,9797 Illinois Rte 76,Belvidere,61008,Boone,16,Adam Kinzinger,69,Sosnowski Joe,...,In Use,0,0,330,1,0,0,Storage,NaN,NaN
8859,Department of Transportation,Quincy Maintenance Storage Facility,800 Koch's Lane,Quincy,62305,Adams,18,Darin M. LaHood,94,Frese Randy E.,...,In Use,0,1987,130,1,0,0,Storage,High Hazard,NaN
8860,Illinois Community College Board,Illinois Valley Community College - Oglesby,815 North Orlando Smith Avenue,Oglesby,61348,LaSalle,16,Adam Kinzinger,76,Long Jerry Lee,...,In Use,1971,1971,49552,1,1,0,Education,Education,Not provided


The first thing I did was transform my data to change 0 values to NaN.

In [4]:
b = pd.read_csv("building_inventory.csv",  
           na_values = {'Square Footage': 0,
                       'Year Acquired': 0,
                       'Year Constructed': 0})

I initially was extremely lost on how to pull three values together. But the TA suggested I create a Pivot Table as a reference table with Square Footage summed up in each value.

In [5]:
b_table = pd.pivot_table(b, index=["Congress Dist"],
                                 values=["Square Footage"],
                                 columns=["Agency Name"],
                                  aggfunc=np.sum)
b_table

Square Footage  \
Agency Name   Appellate Court / Fifth District   
Congress Dist                                    
0                                          NaN   
1                                          NaN   
2                                          NaN   
3                                          NaN   
4                                          NaN   
5                                          NaN   
6                                          NaN   
7                                          NaN   
8                                          NaN   
9                                          NaN   
10                                         NaN   
11                                         NaN   
12                                     15124.0   
13                                         NaN   
14                                         NaN   
15                                         NaN   
16                                         NaN   
17                                         NaN   
18                                         NaN   

                                                 \
Agency Name   Appellate Court / Fourth District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                           NaN   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                      16400.0   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                 \
Agency Name   Appellate Court / Second District   
Congress Dist                                     
0                                           NaN   
1                                           NaN   
2                                           NaN   
3                                           NaN   
4                                           NaN   
5                                           NaN   
6                                           NaN   
7                                           NaN   
8                                       43330.0   
9                                           NaN   
10                                          NaN   
11                                          NaN   
12                                          NaN   
13                                          NaN   
14                                          NaN   
15                                          NaN   
16                                          NaN   
17                                          NaN   
18                                          NaN   

                                                                         \
Agency Name   Appellate Court / Third District Chicago State University   
Congress Dist                                                             
0                                          NaN                      NaN   
1                                          NaN                1219492.0   
2                                          NaN                      NaN   
3                                          NaN                      NaN   
4                                          NaN                      NaN   
5                                          NaN                      NaN   
6         

# Heat Map Construction

In [6]:
#rowname = b_table.index
#columnname = b_table.columns
#valuename = b_table.values

Initially, I used the values above but columnname variable did not work. I needed to transform it into a list.  Also, I had to use log normalization because the values of Total Square Footage had a very large range.

In [7]:
rowname = b_table.index
columnname = b_table.columns.levels[1].to_list()
valuename = np.log10(b_table.values)

## Static Heat Map
I first made an initial heat map without any interactions as a test. I experimented with other scales but Ordinal Scales seemed to work best. It was also initially very confusing that the x axis held column values and the y axis held row values.  

I used the `Reds` color scheme becauuse it was pretty simple and I feel like it's easier to tell differences in value when only one color is used.

In [8]:
# 2. Scales
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()
c_sc = bqplot.ColorScale(scheme='Reds')

# 3. Axes
x_ax = bqplot.Axis(scale=x_sc, label ='Agency Name')
y_ax = bqplot.Axis(scale=y_sc, label = 'Congress Dist', orientation = 'vertical')
c_ax = bqplot.ColorAxis(scale=c_sc, label='Total Square Footage',
                       orientation = 'vertical', side = 'right')

# 4. Marks
heat_map = bqplot.GridHeatMap(color = valuename, row = rowname , column = columnname,
                            scales = {'color':c_sc, 'row':y_sc, 'column':x_sc},)

# FIGURE
fig = bqplot.Figure(marks=[heat_map], axes = [c_ax, y_ax, x_ax])
fig

Figure(axes=[ColorAxis(label='Total Square Footage', orientation='vertical', scale=ColorScale(scheme='Reds'), …

## Interactions for the Heatmap

In [9]:
# (I) HEAT MAP

# 2. Scales
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()
c_sc = bqplot.ColorScale(scheme='Reds')

# 3. Axes
x_ax = bqplot.Axis(scale=x_sc, label ='Agency Name')
y_ax = bqplot.Axis(scale=y_sc, label = 'Congress Dist', orientation = 'vertical')
c_ax = bqplot.ColorAxis(scale=c_sc, label='Total Square Footage',
                       orientation = 'vertical', side = 'right')

# 4. Marks
heat_map = bqplot.GridHeatMap(color = valuename, row = rowname , column = columnname,
                            scales = {'color':c_sc, 'row':y_sc, 'column':x_sc},
                             interactions={'click':'select'}, 
                             anchor_style={'fill':'blue'},
                             selected_style={'opacity':1.0}, 
                             unselected_style={'opacity':0.8}) 

# 5. Interactions
mySelectedLabel = ipywidgets.Label()

def on_selected(change):
    #print(change['owner'].selected)
    if len(change['owner'].selected) == 1:
        i, j = change['owner'].selected[0]
        v = valuename[i,j]
        mySelectedLabel.value = 'Sum of Total Sq Footage =' + str(v)

heat_map.observe(on_selected, 'selected')

# FIGURE
fig = bqplot.Figure(marks=[heat_map], axes = [c_ax, y_ax, x_ax])
myDashboard = ipywidgets.VBox([mySelectedLabel, fig])
myDashboard

# Line Plot

## Region Mask Fiasco
I kept my original attempt at making the region mask below. I initially tried using the original table exclusively until I realized it didn't have any information on time. I then swapped back to using the original data, but was too caught up in copying the format from our in class notes. In the end this single step took me 5 hours and two trips to office hours. But it was a learning experience.

In [66]:
# rowname = b_table.index
# columnname = b_table.columns.levels[1].to_list()
# valuename = np.log10(b_table.values)

In [125]:
# 1. Data and Region Mask
# i,j=5,5
# rowcount = [rowname[i], rowname[i+1]]
# colcount = [columnname[j], columnname[j+1]]

#region_mask = ( (b_table.index >= rowcount[0]) & (b_table.index <= rowcount[1]) &\
 #             (b_table.columns.levels[1].to_list() == colcount[:]))

# region_mask = ( (b_table.index >= rowcount[0]) & (b_table.index <= rowcount[1]) &\
#               (colcount[:] in b_table.columns.levels[1].to_list()))

# partial_mask = ((b_table.index >= rowcount[0]) & (b_table.index <= rowcount[1]))
# partial_mask2 = b_table.query('Agency Name' == 'Square Footage, colcount[0]')

In [225]:
#i,j = 5,5
#distrow = [b[j], b.index[j+1]]
#agcol = [b.columns[i], b.columns[i+1]]
#region_mask = ((b['Congress Dist'] >= distrow[0] & b['Congress Dist'] <= distrow[1] &\
#                b['Agency Name'] is agcol[0] or b['Agency Name'] is agcol[1]))

I was finally able to create my region mask using a hard-coded point on the heat map. The answer was so simple it's actually frustating.

In [179]:
i,j = 12,10

distnum = b_table.index[i]
agency = b_table.columns.levels[1][j]

region_mask = (b['Congress Dist'] == distnum) & (b['Agency Name'] == agency)

## Static Line Plot 
A linear scale worked and I decided to not use a log transformation for the plot since I like how it showed dramatic increases in total square footage without making the smaller increases too small to see. I also did not set consistent y ranges since there was a large range in Year Constructed which would make the graph oddly small in certain subsets.

In [207]:
# 1. Data
subset = b[region_mask].groupby('Year Constructed')['Square Footage'].sum()
xvalue = subset.index
yvalue = subset.values

In [210]:
# 2. Scales
x_scl = bqplot.LinearScale()
y_scl = bqplot.LinearScale()

# 3. Axis
x_axl = bqplot.Axis(label = 'Year Constructed', scale=x_scl)
y_axl = bqplot.Axis(label = 'Total Square Footage Acquired per Year', 
                    scale=y_scl, 
                    orientation='vertical', 
                    side='left')

#4. Mark
line_plot = bqplot.Lines(x=xvalue, y=yvalue, scales={'x':x_scl, 'y':y_scl})

figl = bqplot.Figure(marks=[line_plot], axes=[x_axl, y_axl])
figl

Figure(axes=[Axis(label='Year Constructed', scale=LinearScale()), Axis(label='Total Square Footage Acquired pe…

# Combining It all Together
First used `line_plot.keys` to determine what values I could manipulate to connect the line plot to the heat map.

In [212]:
#line_plot.keys

In [221]:
# (I) HEAT MAP

# 2. Scales
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()
c_sc = bqplot.ColorScale(scheme='Reds')

# 3. Axes
x_ax = bqplot.Axis(scale=x_sc, label ='Agency Name')
y_ax = bqplot.Axis(scale=y_sc, label = 'Congress Dist', orientation = 'vertical')
c_ax = bqplot.ColorAxis(scale=c_sc, label='Total Square Footage',
                       orientation = 'vertical', side = 'right')

# 4. Marks
heat_map = bqplot.GridHeatMap(color = valuename, row = rowname , column = columnname,
                            scales = {'color':c_sc, 'row':y_sc, 'column':x_sc},
                             interactions={'click':'select'}, 
                             anchor_style={'fill':'blue'},
                             selected_style={'opacity':1.0}, 
                             unselected_style={'opacity':0.8}) 

# 5. Interactions
mySelectedLabel = ipywidgets.Label()

def on_selected(change):
    #print(change['owner'].selected)
    if len(change['owner'].selected) == 1:
        i, j = change['owner'].selected[0]
        v = valuename[i,j]
        mySelectedLabel.value = 'Sum of Total Sq Footage =' + str(v)

heat_map.observe(on_selected, 'selected')

# FIGURE
fig = bqplot.Figure(marks=[heat_map], axes = [c_ax, y_ax, x_ax])
#myDashboard = ipywidgets.VBox([mySelectedLabel, fig])
#myDashboard

I had to hash out the region mask and bin functions below since it was preventing my linking function from working.

In [222]:
# (II) LINE PLOT

# 1. Data

# 2. Scales
x_scl = bqplot.LinearScale()
y_scl = bqplot.LinearScale()

# 3. Axis
x_axl = bqplot.Axis(label = 'Year Constructed', scale=x_scl)
y_axl = bqplot.Axis(label = 'Total Square Footage Acquired per Year', 
                    scale=y_scl, 
                    orientation='vertical', 
                    side='left')
# 4. Mark
# 4.1 Bin Selection
#i,j = 12,10

# 4.2 district number and agency name bins
distnum = b_table.index[i]
agency = b_table.columns.levels[1][j]


# 4.3 Region Mask and Subset Values
#region_mask = (b['Congress Dist'] == distnum) & (b['Agency Name'] == agency)
#subset = b[region_mask].groupby('Year Constructed')['Square Footage'].sum()
#xvalue = subset.index
#yvalue = subset.values

#4.4 Line Plot using Region Mask
line_plot = bqplot.Lines(x=xvalue, y=yvalue, scales={'x':x_scl, 'y':y_scl})

figl = bqplot.Figure(marks=[line_plot], axes=[x_axl, y_axl])
#figl

## Linking Function
I created a linking function below by copy and pasting the binning/masking functions from the Line Plot section. 

In [223]:
# (III) LINKING HEATMAP AND LINE PLOT
def on_selected(change):
    if len(change['owner'].selected) == 1:
        i, j = change['owner'].selected[0]
        v = valuename[i,j]
        mySelectedLabel.value = 'Sum of Total Sq Footage =' + str(v)
        #i,j = 12,10
        distnum = b_table.index[i]
        agency = b_table.columns.levels[1][j]
        region_mask = (b['Congress Dist'] == distnum) & (b['Agency Name'] == agency)
        subset = b[region_mask].groupby('Year Constructed')['Square Footage'].sum()
        line_plot.x = subset.index
        line_plot.y = subset.values
        
heat_map.observe(on_selected,'selected')

## Heat Map and Line Plot linked

In [227]:
fig.layout.min_width='500px'
figl.layout.min_width='500px'
figures = ipywidgets.HBox([fig, figl])
myDashboard = ipywidgets.VBox([mySelectedLabel, figures])
myDashboard

# Aesthetic Changes
I edited some parts of the dashboard to the best of my ability in order to make it easier to understand and a bit cleaner.
- Shifted y-axis label on line plot away from the figure using `label_offset`.
- Shift x-axis label on heat map away from the figure using `label_offset`.
- Rotated x-axis ticks using `tick_rotate` and changed font size using `tick_style`.
- Abbreviated color scale label to help it fit better.

In [292]:
# (I) HEAT MAP

# 2. Scales
x_sc = bqplot.OrdinalScale()
y_sc = bqplot.OrdinalScale()
c_sc = bqplot.ColorScale(scheme='Reds')

# 3. Axes
x_ax = bqplot.Axis(scale=x_sc, label ='Agency Name', 
                   tick_rotate = 90, 
                   tick_style={'font-size': 8},
                  label_offset='55px')
y_ax = bqplot.Axis(scale=y_sc, label = 'Congress Dist', orientation = 'vertical')
c_ax = bqplot.ColorAxis(scale=c_sc, label='Total Sq Ft',
                       orientation = 'vertical', side = 'right')

# 4. Marks
heat_map = bqplot.GridHeatMap(color = valuename, row = rowname , column = columnname,
                            scales = {'color':c_sc, 'row':y_sc, 'column':x_sc},
                             interactions={'click':'select'}, 
                             anchor_style={'fill':'blue'},
                             selected_style={'opacity':1.0}, 
                             unselected_style={'opacity':0.8}) 

# 5. Interactions
mySelectedLabel = ipywidgets.Label()

def on_selected(change):
    #print(change['owner'].selected)
    if len(change['owner'].selected) == 1:
        i, j = change['owner'].selected[0]
        v = valuename[i,j]
        mySelectedLabel.value = 'Sum of Total Sq Footage =' + str(v)

heat_map.observe(on_selected, 'selected')

# FIGURE
fig = bqplot.Figure(marks=[heat_map], axes = [c_ax, y_ax, x_ax])
#myDashboard = ipywidgets.VBox([mySelectedLabel, fig])
#myDashboard

In [293]:
# (II) LINE PLOT

# 1. Data

# 2. Scales
x_scl = bqplot.LinearScale()
y_scl = bqplot.LinearScale()

# 3. Axis
x_axl = bqplot.Axis(label = 'Year Constructed', scale=x_scl)
y_axl = bqplot.Axis(label = 'Total Square Footage Acquired per Year',
                    label_offset='50px',
                    scale=y_scl, 
                    orientation='vertical', 
                    side='left')
# 4. Mark
# 4.1 Bin Selection
#i,j = 12,10

# 4.2 district number and agency name bins
distnum = b_table.index[i]
agency = b_table.columns.levels[1][j]


# 4.3 Region Mask and Subset Values
#region_mask = (b['Congress Dist'] == distnum) & (b['Agency Name'] == agency)
#subset = b[region_mask].groupby('Year Constructed')['Square Footage'].sum()
#xvalue = subset.index
#yvalue = subset.values

#4.4 Line Plot using Region Mask
line_plot = bqplot.Lines(x=xvalue, y=yvalue, scales={'x':x_scl, 'y':y_scl})

figl = bqplot.Figure(marks=[line_plot], axes=[x_axl, y_axl])
#figl

In [294]:
# (III) LINKING HEATMAP AND LINE PLOT
def on_selected(change):
    if len(change['owner'].selected) == 1:
        i, j = change['owner'].selected[0]
        v = valuename[i,j]
        mySelectedLabel.value = 'Sum of Total Sq Footage =' + str(v)
        #i,j = 12,10
        distnum = b_table.index[i]
        agency = b_table.columns.levels[1][j]
        region_mask = (b['Congress Dist'] == distnum) & (b['Agency Name'] == agency)
        subset = b[region_mask].groupby('Year Constructed')['Square Footage'].sum()
        line_plot.x = subset.index
        line_plot.y = subset.values
        
heat_map.observe(on_selected,'selected')

# Final Product

In [296]:
fig.layout.min_width='400px'
figl.layout.min_width='400px'
figures = ipywidgets.HBox([fig, figl])
myDashboard = ipywidgets.VBox([mySelectedLabel, figures])
myDashboard